In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce

### 匯入資料
- 資料時間 : 2011 ~ 2020
- 預計欄位
 - 香蕉 前一日價格
 - 香蕉 平均價格
 - 香蕉 五日平均價格
 - 香蕉 成交量
 - 芭蕉 前日價格
 - 芭樂 前日價格
 - 節日(暫不納入)
 

#### 香蕉

In [6]:
df_banana = pd.read_csv("./cleaned data/價格_全國香蕉交易資料(初步清洗).csv", usecols=[0,1,2,6,7])

df_banana.head(10)

,日期,市場,產品,平均價(元/公斤),交易量(公斤)
0,2011-01-01,104 台北二,A1 香蕉,31.4,9645
1,2011-01-02,104 台北二,A1 香蕉,32.0,12230
2,2011-01-04,104 台北二,A1 香蕉,32.4,11976
3,2011-01-05,104 台北二,A1 香蕉,33.2,7520
4,2011-01-06,104 台北二,A1 香蕉,34.7,8160
5,2011-01-07,104 台北二,A1 香蕉,35.5,7152
6,2011-01-08,104 台北二,A1 香蕉,36.2,5850
7,2011-01-09,104 台北二,A1 香蕉,35.4,8235
8,2011-01-11,104 台北二,A1 香蕉,35.4,7320
9,2011-01-12,104 台北二,A1 香蕉,35.7,5730


In [7]:
df_banana.groupby("市場").sum()["交易量(公斤)"].sort_values(ascending=False)

市場
400 台中市    98946562
109 台北一    64851863
241 三重區    58371868
104 台北二    41543316
830 鳳山區    27701450
420 豐原區    15670654
800 高雄市    11969933
220 板橋區    10649034
600 嘉義市     6568129
260 宜蘭市     4708040
930 台東市     2437560
338 桃農       153565
423 東勢區       39878
Name: 交易量(公斤), dtype: int64

In [8]:
## 台北一市場
df_banana_TP = df_banana[df_banana["市場"] == "109 台北一"]
df_banana_TP.columns = ['日期', '市場', '產品', '香蕉_平均價', '香蕉_交易量', 'Year']
df_banana_TP.head()

ValueError: Length mismatch: Expected axis has 5 elements, new values have 6 elements

In [9]:
# 新欄位
df_banana_TP["香蕉_5日平均價格"] = round(df_banana_TP["香蕉_平均價"].rolling(5).mean().fillna(method="bfill"), 1)
df_banana_TP["香蕉_前1日平均價格"] = df_banana_TP["香蕉_平均價"].shift(1).fillna(method="bfill")

KeyError: '香蕉_平均價'

In [10]:
df_banana_TP["日期"] = pd.to_datetime(df_banana_TP["日期"])

<ipython-input-10-e04fc00c08a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_banana_TP["日期"] = pd.to_datetime(df_banana_TP["日期"])


In [11]:
# index設為日期
df_banana_TP = df_banana_TP.set_index("日期")
df_banana_TP.count()

市場           3145
產品           3145
平均價(元/公斤)    3145
交易量(公斤)      3145
dtype: int64

In [ ]:
# resample
df_banana_TP = df_banana_TP.resample("D").ffill()
df_banana_TP.head()


#### 芭蕉

In [ ]:
df_SecretBanana = pd.read_csv("../data/farmproduct_芭蕉_crawler.csv", usecols=[1,2,3,7,9])

In [ ]:
target_cols = ['市場', '產品','平均價(元/公斤)','交易量(公斤)']
df_SecretBanana[target_cols] = df_SecretBanana[target_cols].apply(lambda x: x.str.strip(" "))
df_SecretBanana = df_SecretBanana[~(df_SecretBanana["市場"] == "A2 香蕉 芭蕉紅芭蕉")]
df_SecretBanana.head()

In [ ]:
df_SecretBanana["日期"] = df_SecretBanana["日期"].apply(lambda x: re.sub("\d{3}", "{}".format((int(x.split("/")[0]) + 1911)), x))
df_SecretBanana["平均價(元/公斤)"] = df_SecretBanana["平均價(元/公斤)"].astype("float")
df_SecretBanana["交易量(公斤)"] = df_SecretBanana["交易量(公斤)"].apply(lambda x:re.sub(",","",x)).astype("int")

In [ ]:
### 台北一市場
# 筆數較多是因為爬蟲時間不一樣所致
df_SecretBanana_TP = df_SecretBanana[df_SecretBanana["市場"] == "109 台北一"]
df_SecretBanana_TP = df_SecretBanana_TP[["日期", "平均價(元/公斤)"]]


In [ ]:
df_SecretBanana_TP["日期"] = pd.to_datetime(df_SecretBanana_TP["日期"])
df_SecretBanana_TP.set_index("日期",inplace=True)
df_SecretBanana_TP["平均價(元/公斤)"] = df_SecretBanana_TP["平均價(元/公斤)"].shift(1).fillna(method="bfill")
                                                                       

In [ ]:
df_SecretBanana_TP.columns = ["芭蕉_前1日平均價"]

In [ ]:
df_SecretBanana_TP = df_SecretBanana_TP.resample("D").ffill()
df_SecretBanana_TP.head()

### 芭樂

In [ ]:
df_grava = pd.read_csv("../data/farmproduct_grava_crawler.csv", usecols=[0,1,6])
df_grava.head()

In [ ]:
df_grava_TP = df_grava[df_grava["市場"] == "109 台北一"]
df_grava_TP["日期"] = df_grava_TP["日期"].apply(lambda x: re.sub("\d{3}", "{}".format((int(x.split("/")[0]) + 1911)), x))
df_grava_TP["平均價(元/公斤)"] = df_grava_TP["平均價(元/公斤)"].shift(1).fillna(method="bfill") 

df_grava_TP["日期"] = pd.to_datetime(df_grava_TP["日期"])
df_grava_TP.set_index("日期", inplace=True)

df_grava_TP.drop(columns="市場",inplace=True)
df_grava_TP.columns = ["芭樂_前1日平均價格"]
df_grava_TP.head()

In [ ]:
df_grava_TP = df_grava_TP.resample("D").ffill()
df_grava_TP.head()

## 合併dataframes


In [ ]:
df_banana_TP.head()

In [ ]:
df_SecretBanana_TP.head()

In [ ]:
df_grava_TP.head()

In [ ]:
dfs = [df_banana_TP, df_SecretBanana_TP, df_grava_TP]
df_merged = reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True), dfs)
df_merged.head()

In [ ]:
df_merged.drop(columns="Year", inplace=True)

In [ ]:
df_merged.to_csv("分析資料(0722).csv")